# Learning and Decision Making

## Laboratory 4: Reinforcement learning

In the end of the lab, you should export the notebook to a Python script (``File >> Download as >> Python (.py)``). Make sure that the resulting script includes all code written in the tasks marked as "**Activity n. N**", together with any replies to specific questions posed. Your file should be named `padi-labKK-groupXXX.py`, where `KK` corresponds to the lab number and the `XXX` corresponds to your group number. Similarly, your homework should consist of a single pdf file named `padi-hwKK-groupXXX.pdf`. You should create a zip file with the lab and homework files and submit it in Fenix **at most 30 minutes after your lab is over**.

Make sure to strictly respect the specifications in each activity, in terms of the intended inputs, outputs and naming conventions.

In particular, after completing the activities you should be able to replicate the examples provided (although this, in itself, is no guarantee that the activities are correctly completed).

### 1. The MDP Model 

In this lab you will implement several reinforcement learning algorithms, and use the taxi domain from Lab 2 to test and compare these algorithms. Don't forget, however, that your functions should work for **any MDP** and not just the one provided. 

The taxi domain to be used is represented in the diagram below.

<img src="taxi.png" width="250px">

In the above domain, 

* The taxi can be in any of the 25 cells in the diagram. The passenger can be at any of the 4 marked locations ($Y$, $B$, $G$, $R$) or in the taxi. Additionally, the passenger wishes to go to one of the 4 possible destinations. The total number of states, in this case, is $25\times 5\times 4$.
* At each step, the agent (taxi driver) may move in any of the four directions -- south, north, east and west. It can also pickup the passenger or drop off the passenger. 
* The goal of the taxi driver is to pickup the passenger and drop it at the passenger's desired destination.

**Throughout the lab, unless if stated otherwise, use $\gamma=0.99$.**

$$\diamond$$

We start by loading the MDP for the taxi domain from the file `taxi.npz`. We will use this domain as an example to illustrate the different functions/algorithms you are expected to deploy. The file contains both the MDP, described as a tuple like those from Lab 2, and the corresponding optimal $Q$-function.

To do so, you can run the code
```python
import numpy as np

mdp_info = np.load('taxi.npz', allow_pickle=True)

# The MDP is a tuple (X, A, P, c, gamma)
M = tuple(mdp_info['M'])

# We also load the optimal Q-function for the MDP
Qopt = mdp_info['Q']
```

---

In the first activity, you will implement a "simulator of the world". The simulator consists of a function that enables you to sample a transition from a given MDP. You will then use this function, in subsequent activities, to generate the data that your agent will use to learn.

---

#### Activity 1.        

Write a function named `sample_transition` that receives, as input, a tuple representing an arbitrary MDP as well as two integers, `x` and `a`, corresponding to a state and an action. The function should return a tuple `(x, a, c, x')`, where `c` is the cost associated with performing action `a` in state `x` and `x'` is a state generated from `x` upon selecting action `a`, according to the transition probabilities for the MDP.

---

In [62]:
import numpy as np

mdp_info = np.load('taxi.npz', allow_pickle=True)

# The MDP is a tuple (X, A, P, c, gamma)
M = tuple(mdp_info['M'])

# We also load the optimal Q-function for the MDP
Qopt = mdp_info['Q']

In [63]:
import numpy as np

def sample_transition(mdp:tuple, x:int, a:int):
    X, A, P, c, gamma = mdp 
    cost = c[x][a]
    x_new = rnd.choice(np.where(P[a][x] == np.max(P[a][x]))[0])
    return x, a, cost, x_new


import numpy.random as rnd

rnd.seed(42)

# Select random state and action
x = 175 # State (9, B, B)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('Observed transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')

# Select random state and action
x = 187 # State (10, G, B)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('\nObserved transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')

# Select random state and action
x = 69 # State (4, Y, G)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('\nObserved transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')

Observed transition:
((9, B, B), West, 0.7, (8, B, B))

Observed transition:
((10, G, B), Pickup, 1.0, (10, G, B))

Observed transition:
((4, Y, G), East, 0.7, (5, Y, G))


All reinforcement learning algorithms that you will implement can only access the MDP through the function `sample_transition` which, in a sense, simulates an "interaction" of the agent with the environment.

For example, using the taxi MDP, you could run:

```python
import numpy.random as rnd

rnd.seed(42)

# Select random state and action
x = 175 # State (9, B, B)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('Observed transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')

# Select random state and action
x = 187 # State (10, G, B)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('\nObserved transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')

# Select random state and action
x = 69 # State (4, Y, G)
a = rnd.randint(len(M[1]))

x, a, cnew, xnew = sample_transition(M, x, a)

print('\nObserved transition:\n(', end='')
print(M[0][x], end=', ')
print(M[1][a], end=', ')
print(cnew, end=', ')
print(M[0][xnew], end=')\n')
```

and get, as output:

```
Observed transition:
((9, B, B), West, 0.7, (8, B, B))

Observed transition:
((10, G, B), East, 0.7, (10, G, B))

Observed transition:
((4, Y, G), Pickup, 1.0, (4, Y, G))
```

**Note:** For debug purposes, we also provide a second file, `taxi-small.npz`, that contains a 9-state MDP that you can use to verify if your results make sense.

---

#### Activity 2.        

Write down a function named `egreedy` that implements an $\epsilon$-greedy policy. Your function should receive, as input, a `numpy` array `Q` with shape `(N,)`, for some integer `N`, and, as an optional argument, a floating point number `eps` with a default value `eps=0.1`. Your function should return... 

* ... with a probability $\epsilon$, a random index between $0$ and $N-1$.
* ... with a probability $1-\epsilon$, the index between $0$ and $N-1$ corresponding to the minimum value of `Q`. If more than one such index exists, the function should select among such indices **uniformly at random**.

**Note:** In the upcoming activities, the array `Q` received by the function `egreedy` will correspond to a row of a $Q$-function, and `N` will correspond to the number of actions.

In [64]:
import numpy as np

def egreedy(Q, eps: float=0.1):
    if rnd.random() < eps:
        return rnd.choice(range(len(Q)))
    else:
        return rnd.choice(np.where(Q == np.min(Q))[0])

rnd.seed(42)

x = 175 # State (9, B, B)
a = egreedy(Qopt[x, :], eps=0)
print('State:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])

x = 187 # State (10, G, B)
a = egreedy(Qopt[x, :], eps=0)
print('\nState:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])

x = 69 # State (4, Y, G)
a = egreedy(Qopt[x, :], eps=0)
print('\nState:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])

State: (9, B, B) - action (eps=0.0): South
State: (9, B, B) - action (eps=0.5): South
State: (9, B, B) - action (eps=1.0): Pickup

State: (10, G, B) - action (eps=0.0): North
State: (10, G, B) - action (eps=0.5): East
State: (10, G, B) - action (eps=1.0): Pickup

State: (4, Y, G) - action (eps=0.0): West
State: (4, Y, G) - action (eps=0.5): South
State: (4, Y, G) - action (eps=1.0): West


For example, using the function `Qopt` loaded from the taxi file, you can run:

```python
rnd.seed(42)

x = 175 # State (9, B, B)
a = egreedy(Qopt[x, :], eps=0)
print('State:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])

x = 187 # State (10, G, B)
a = egreedy(Qopt[x, :], eps=0)
print('\nState:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])

x = 69 # State (4, Y, G)
a = egreedy(Qopt[x, :], eps=0)
print('\nState:', M[0][x], '- action (eps=0.0):', M[1][a])
a = egreedy(Qopt[x, :], eps=0.5)
print('State:', M[0][x], '- action (eps=0.5):', M[1][a])
a = egreedy(Qopt[x, :], eps=1.0)
print('State:', M[0][x], '- action (eps=1.0):', M[1][a])
```

and you will get the output:

```
State: (9, B, B) - action (eps=0.0): South
State: (9, B, B) - action (eps=0.5): South
State: (9, B, B) - action (eps=1.0): East

State: (10, G, B) - action (eps=0.0): North
State: (10, G, B) - action (eps=0.5): East
State: (10, G, B) - action (eps=1.0): North

State: (4, Y, G) - action (eps=0.0): West
State: (4, Y, G) - action (eps=0.5): West
State: (4, Y, G) - action (eps=1.0): West
```

**Note that, depending on the order and number of calls to functions in the random library you may get slightly different results.**

---

#### Activity 3. 

Write a function `mb_learning` that implements the model-based reinforcement learning algorithm discussed in class. Your function should receive as input arguments 

* A tuple, `mdp`, containing the description of an **arbitrary** MDP. The structure of the tuple is similar to that provided in the example above. 
* An integer, `n`, corresponding the number of steps that your algorithm should run.
*  A numpy array `qinit` with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`. The matrix `qinit` should be used to initialize the $Q$-function being learned by your function.
* A tuple, `Pinit`, with as many elements as the number of actions in `mdp`. Each element of `Pinit` corresponds to square numpy arrays with as many rows/columns as the number of states in `mdp` and can be **any** transition probability matrix. The matrices in `Pinit` should be used to initialize the transition probability matrices of the model being learned by your function.
* A numpy array `cinit` with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`. The matrix `cinit` should be used to initialize the cost function of the model being learned by your function.

Your function should simulate an interaction of `n` steps between the agent and the environment, starting from an arbitrary state chosen uniformly at random, and during which it should perform `n` iterations of the model-based RL algorithm seen in class. In particular, it should learn the transition probabilities and cost function from the interaction between the agent and the environment, and use these to compute the optimal $Q$-function. The transition probabilities, cost and $Q$-functions to be learned should be initialized using `Pinit`, `cinit` and `qinit`, respectively. 

Note that, at each step of the interaction,

* The agent should observe the current state, and select an action using an $\epsilon$-greedy policy with respect to its current estimate of the optimal $Q$-values. You should use the function `egreedy` from Activity 2, with $\epsilon=0.15$. 
* Given the state and action, you must then compute the cost and generate the next state, using `mdp` and the function `sample_transition` from Activity 1.
* With this transition information (state, action, cost, next-state), you can now perform an update to the transition probabilities, cost function, and $Q$-function.
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

Your function should return a tuple containing:

*  A numpy array with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`, corresponding to the learned $Q$-function.
* A tuple with as many elements as the number of actions in `mdp`. The element $a$ of the tuple corresponds to a square numpy array with as many rows/columns as the number of states in `mdp`, corresponding to the learned transition probabilities for action $a$.
* A numpy array with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`, corresponding to the learned cost function.

---

In [92]:
import numpy as np
import numpy.random as rnd
def mb_learning(mdp:tuple, n:int, qinit:np.array, Pinit:tuple, cinit:np.array):
    X, A, P_mdp, c_mdp, gamma = mdp
    x = rnd.randint(len(X))
    N = len(A)
    Q = qinit
    P = Pinit
    c = cinit
    N_t = np.zeros((len(X), N))
    for _ in range(n):
        a = egreedy(Q[x], 0.15)
        _, _, cost, x_new= sample_transition(mdp, x, a)

        alpha = 1 / (N_t[x][a] + 1)
        P += alpha * (np.eye(len(X))[x_new] - P)
        c += alpha * (cost - c) 

        N_t[x][a] += 1

        Q_next_min = np.min(Q, axis=1)
        Q_next = np.dot(P[a][x], Q_next_min)
        
        # Update Q-values using the provided equation
        Q[x][a] = c[x][a] + gamma * np.sum(Q_next)
        x = x_new
    return Q, tuple(P), c


rnd.seed(42)

# Initialize transition probabilities
pinit = ()

for a in range(len(M[1])):
    pinit += (np.eye(len(M[0])),)

# Initialize cost function
cinit = np.zeros((len(M[0]), len(M[1])))

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew, pnew, cnew = mb_learning(M, 1000, qinit, pinit, cinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew, pnew, cnew = mb_learning(M, 1000, qnew, pnew, cnew)

# Compare once again the learned Q with the optimal Q
print('Error in Q after 2000 steps:', np.linalg.norm(qnew - Qopt))

Error in Q after 1000 steps: 388.72130940745296
Error in Q after 2000 steps: 378.4927134561178


As an example using the taxi MDP, we could run:

```python
rnd.seed(42)

# Initialize transition probabilities
pinit = ()

for a in range(len(M[1])):
    pinit += (np.eye(len(M[0])),)

# Initialize cost function
cinit = np.zeros((len(M[0]), len(M[1])))

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew, pnew, cnew = mb_learning(M, 1000, qinit, pinit, cinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew, pnew, cnew = mb_learning(M, 1000, qnew, pnew, cnew)

# Compare once again the learned Q with the optimal Q
print('Error in Q after 2000 steps:', np.linalg.norm(qnew - Qopt))
```

to get

```
Error in Q after 1000 steps: 388.8249426163114
Error in Q after 2000 steps: 388.8221418599741
```

Note that, even if the seed is fixed, the numerical values may differ somewhat from those above.

### 3. Model-free learning

You will now implement both $Q$-learning and SARSA.

---

#### Activity 4. 

Write a function `qlearning` that implements the $Q$-learning algorithm discussed in class. Your function should receive as input arguments 

* A tuple, `mdp`, containing the description of an **arbitrary** MDP. The structure of the tuple is similar to that provided in the examples above. 
* An integer, `n`, corresponding he number of steps that your algorithm should run.
*  A `numpy` array `qinit` with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`. The matrix `qinit` should be used to initialize the $Q$-function being learned by your function.

Your function should simulate an interaction of `n` steps between the agent and the environment, starting from an arbitrary state chosen uniformly at random, and during which it should perform `n` iterations of the $Q$-learning algorithm seen in class. In particular, it should learn optimal $Q$-function. The $Q$-function to be learned should be initialized using `qinit`. 

Note that, at each step of the interaction,

* The agent should observe the current state, and select an action using an $\epsilon$-greedy policy with respect to its current estimate of the optimal $Q$-values. You should use the function `egreedy` from Activity 2, with $\epsilon=0.15$. 
* Given the state and action, you must then compute the cost and generate the next state, using `mdp` and the function `sample_transition` from Activity 1.
* With this transition information (state, action, cost, next-state), you can now perform an update to the $Q$-function.
* When updating the components $(x,a)$ of the model, use the step-size $\alpha=0.3$.

Your function should return a `numpy` array with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`, corresponding to the learned $Q$-function.

---

In [91]:
def qlearning(mdp: tuple, n: int, qinit: np.ndarray):
    X, A, _, _, gamma = mdp

    Q = qinit.copy()
    
    x = rnd.randint(len(X))
    
    for _ in range(n):
        
        a = egreedy(Q[x], eps=0.15)
        
        x, a, cost, x_prime = sample_transition(mdp, x, a)
        
        alpha = 0.3
        Q[x, a] = Q[x, a] + alpha * (cost + gamma * np.max(Q[x_prime]) - Q[x, a])
        
        x = x_prime
    
    return Q


rnd.seed(42)

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew = qlearning(M, 1000, qinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew = qlearning(M, 1000, qnew)

for i in range(500):
    # Compare the learned Q with the optimal Q
    print('Error in Q after',(i+1) * 1000,'steps:', np.linalg.norm(qnew - Qopt))

    # Run 1000 additional steps of model-based learning
    qnew = qlearning(M, 5000, qnew)


Error in Q after 1000 steps: 389.1083239578609
Error in Q after 1000 steps: 388.53818164643576
Error in Q after 2000 steps: 387.27960171929
Error in Q after 3000 steps: 387.26698944320395
Error in Q after 4000 steps: 382.22769426446564
Error in Q after 5000 steps: 381.4659502446264
Error in Q after 6000 steps: 376.8869406039795
Error in Q after 7000 steps: 371.7586335968834
Error in Q after 8000 steps: 363.9284526840498
Error in Q after 9000 steps: 350.93058337861055
Error in Q after 10000 steps: 346.89064043324396
Error in Q after 11000 steps: 342.25757353412934
Error in Q after 12000 steps: 338.95639253960036
Error in Q after 13000 steps: 338.92942619645606
Error in Q after 14000 steps: 334.19570285869946
Error in Q after 15000 steps: 331.0042237682255
Error in Q after 16000 steps: 327.64057973224345
Error in Q after 17000 steps: 319.4432475399161
Error in Q after 18000 steps: 312.1857191407452
Error in Q after 19000 steps: 309.27612489426593
Error in Q after 20000 steps: 309.2761248

Error in Q after 1000 steps: 388.4431476570159
Error in Q after 2000 steps: 388.0174458876989
Error in Q after 3000 steps: 387.4375098536993
Error in Q after 4000 steps: 386.78717191665015
Error in Q after 5000 steps: 382.97135539410607
Error in Q after 6000 steps: 379.29816388372336

As an example using the taxi MDP, we could run:

```python
rnd.seed(42)

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew = qlearning(M, 1000, qinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew = qlearning(M, 1000, qnew)

# Compare once again the learned Q with the optimal Q
print('Error in Q after 2000 steps:', np.linalg.norm(qnew - Qopt))
```

to get

```
Error in Q after 1000 steps: 389.72801262556976
Error in Q after 2000 steps: 389.7261836534418
```

Once again, even if the seed is fixed, the numerical values may differ somewhat from those above.

---

#### Activity 5. 

Write a function `sarsa` that implements the SARSA algorithm discussed in class. Your function should receive as input arguments 

* A tuple, `mdp`, containing the description of an **arbitrary** MDP. The structure of the tuple is similar to that provided in the examples above. 
* An integer, `n`, corresponding he number of steps that your algorithm should run.
*  A `numpy` array `qinit` with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`. The matrix `qinit` should be used to initialize the $Q$-function being learned by your function.

Your function should simulate an interaction of `n` steps between the agent and the environment, starting from an arbitrary state chosen uniformly at random, and during which it should perform `n` iterations of the SARSA algorithm seen in class. The $Q$-function to be learned should be initialized using `qinit`. 

Note that, at each step of the interaction,

* The agent should observe the current state, and select an action using an $\epsilon$-greedy policy with respect to its current estimate of the optimal $Q$-values. You should use the function `egreedy` from Activity 2, with $\epsilon=0.15$. **Do not adjust the value of $\epsilon$ during learning.**
* Given the state and action, you must then compute the cost and generate the next state, using `mdp` and the function `sample_transition` from Activity 1.
* With this transition information (state, action, cost, next-state), you can now perform an update to the $Q$-function.
* When updating the components $(x,a)$ of the model, use the step-size $\alpha=0.3$.

Your function should return a `numpy` array with as many rows as the number of states in `mdp` and as many columns as the number of actions in `mdp`, corresponding to the learned $Q$-function.

---

In [67]:
def sarsa(mdp:tuple, n:int, qinit:np.array):
    X, A, P, c, gamma = mdp
    n_states = len(X)
    x = rnd.randint(n_states)
    Q = qinit
    a = egreedy(Q[x], 0.15)

    for _ in range(n):
        _, _, cost, x_new = sample_transition(mdp, x, a)
        a_new = egreedy(Q[x_new], 0.15)
        Q[x][a] = Q[x][a] + 0.3 * (cost + gamma * Q[x_new][a_new] - Q[x][a])
        x = x_new
        a = a_new
    return Q


rnd.seed(42)

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew = sarsa(M, 1000, qinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew = sarsa(M, 1000, qnew)

# Compare once again the learned Q with the optimal Q
print('Error in Q after 2000 steps:', np.linalg.norm(qnew - Qopt))

Error in Q after 1000 steps: 390.0723574051231
Error in Q after 2000 steps: 389.97599666017805


As an example using the taxi MDP, we could run:

```python
rnd.seed(42)

# Initialize Q-function
qinit = np.zeros((len(M[0]), len(M[1])))

# Run 1000 steps of model-based learning
qnew = sarsa(M, 1000, qinit)

# Compare the learned Q with the optimal Q
print('Error in Q after 1000 steps:', np.linalg.norm(qnew - Qopt))

# Run 1000 additional steps of model-based learning
qnew = sarsa(M, 1000, qnew)

# Compare once again the learned Q with the optimal Q
print('Error in Q after 2000 steps:', np.linalg.norm(qnew - Qopt))
```

to get

```
Error in Q after 1000 steps: 388.44986735098877
Error in Q after 2000 steps: 387.75181889394776
```

In [69]:
%matplotlib inline

import matplotlib.pyplot as plt
from tqdm import trange

STEPS = 100
ITERS = 10000
RUNS  = 5

iters = range(0, STEPS * ITERS + 1, STEPS)

# Error matrices
Emb = np.zeros(ITERS + 1)
Eql = np.zeros(ITERS + 1)
Ess = np.zeros(ITERS + 1)

Emb[0] = np.linalg.norm(Qopt) * RUNS
Eql[0] = Emb[0]
Ess[0] = Emb[0]

rnd.seed(42)

for n in range(RUNS):
    
    print('\n = Training (run n. %i) =' % n)

    # Initialization
    pmb = ()
    for a in range(len(M[1])):
        pmb += (np.eye(len(M[0])),)
    cmb = np.zeros((len(M[0]), len(M[1])))
    qmb = np.zeros((len(M[0]), len(M[1])))

    qql = np.zeros((len(M[0]), len(M[1])))

    qss = np.zeros((len(M[0]), len(M[1])))

    # Run evaluation
    for t in trange(ITERS):
        qmb, pmb, cmb = mb_learning(M, STEPS, qmb, pmb, cmb)
        Emb[t + 1] += np.linalg.norm(Qopt - qmb)

        qql = qlearning(M, STEPS, qql)
        Eql[t + 1] += np.linalg.norm(Qopt - qql)

        qss = sarsa(M, STEPS, qss)
        Ess[t + 1] += np.linalg.norm(Qopt - qss)
        
Emb /= RUNS
Eql /= RUNS
Ess /= RUNS

plt.figure()
plt.plot(iters, Emb, label='Model based learning')
plt.plot(iters, Eql, label='Q-learning')
plt.plot(iters, Ess, label='SARSA')
plt.legend()
plt.xlabel('N. iterations')
plt.ylabel('Error in $Q$-function')
plt.tight_layout()


 = Training (run n. 0) =


  2%|▏         | 244/10000 [08:11<5:27:14,  2.01s/it]


KeyboardInterrupt: 

You can also run the following code, to compare the performance of the three methods:

```python
%matplotlib inline

import matplotlib.pyplot as plt
from tqdm import trange

STEPS = 100
ITERS = 10000
RUNS  = 5

iters = range(0, STEPS * ITERS + 1, STEPS)

# Error matrices
Emb = np.zeros(ITERS + 1)
Eql = np.zeros(ITERS + 1)
Ess = np.zeros(ITERS + 1)

Emb[0] = np.linalg.norm(Qopt) * RUNS
Eql[0] = Emb[0]
Ess[0] = Emb[0]

rnd.seed(42)

for n in range(RUNS):
    
    print('\n = Training (run n. %i) =' % n)

    # Initialization
    pmb = ()
    for a in range(len(M[1])):
        pmb += (np.eye(len(M[0])),)
    cmb = np.zeros((len(M[0]), len(M[1])))
    qmb = np.zeros((len(M[0]), len(M[1])))

    qql = np.zeros((len(M[0]), len(M[1])))

    qss = np.zeros((len(M[0]), len(M[1])))

    # Run evaluation
    for t in trange(ITERS):
        qmb, pmb, cmb = mb_learning(M, STEPS, qmb, pmb, cmb)
        Emb[t + 1] += np.linalg.norm(Qopt - qmb)

        qql = qlearning(M, STEPS, qql)
        Eql[t + 1] += np.linalg.norm(Qopt - qql)

        qss = sarsa(M, STEPS, qss)
        Ess[t + 1] += np.linalg.norm(Qopt - qss)
        
Emb /= RUNS
Eql /= RUNS
Ess /= RUNS

plt.figure()
plt.plot(iters, Emb, label='Model based learning')
plt.plot(iters, Eql, label='Q-learning')
plt.plot(iters, Ess, label='SARSA')
plt.legend()
plt.xlabel('N. iterations')
plt.ylabel('Error in $Q$-function')
plt.tight_layout()
```

**Note:** The code above takes a while to conclude. If you want to observe faster results, you may try with a single run (set `RUNS = 1` above) or decrease the training time (changing `ITERS` above). However, the plot you will obtain will differ from the one provided.

As the output, you should observe a plot similar to the one below.

<img src="plot.png" align="left">

---

#### Activity 6.

**Based on the results you obtained when running the above code with your algorithms**, discuss the differences observed between the performance of the three methods.

---


SARSA (on-policy)
Obtém menores erros mais rapidamente, mas não converge exatamente para a politica optima.
Toma uma rota mais segura devido à probablididade, epsilon, de escolher um movimento aleatório, que o pode levar a cair do penhasco, como representado no caso do cliff walking.

Pode convergir para politica otima se o valor de epsilon for diminuindo ao longo das iterações (faz de cada vez menos random moves)

Q-learning (off-policy)
Necessita de mais iteracoes para convergir, mas converge para a politica otima diretamente.
